In [1]:
from audiocraft.models import MAGNeT
model = MAGNeT.get_pretrained('facebook/magnet-small-10secs')

/Volumes/CrucialX6/2.current/audiocraft/venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Volumes/CrucialX6/2.current/audiocraft/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Volumes/CrucialX6/2.current/audiocraft/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Volumes/CrucialX6/2.current/audiocraft/venv/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated i

In [3]:
import torch
def sample_argmax(tokens, conditions, temp=3):
    logits = model.lm(tokens, conditions=conditions)
    probs = torch.softmax(logits / temp, dim=-1)
    sampled_tokens = torch.argmax(logits, dim=-1, keepdim=True)
    sampled_probs = torch.gather(probs, 3, sampled_tokens)[..., 0]
    return sampled_tokens, sampled_probs


In [33]:
conditions, _ = model._prepare_tokens_and_attributes(['a reggae song'], None)
tokens = (torch.rand((1,4,10), device=model.device) * 1024).long()

conditions, tokens

([ConditioningAttributes(text={'description': 'a reggae song'}, wav={}, joint_embed={})],
 tensor([[[ 277,  337, 1016,  593,  810,  871,  809,  168,  341,  440],
          [ 991,  641,  791,  854, 1006,  607,  437,  302,  495,  111],
          [ 614,  698,  162,  890,   35,    3,  173,   35,  200,  231],
          [ 243,  651,  854,  854,  150,  604,  682,  539,  376,  485]]]))

In [34]:
sampled_tokens, sampled_probs = sample_argmax(tokens, conditions)
sampled_tokens[0][1].t(), sampled_probs[0][1]

(tensor([[ 342, 1697,  156, 1771,  580, 1398, 1888,  522, 1206, 1728]]),
 tensor([0.0023, 0.0021, 0.0031, 0.0024, 0.0019, 0.0026, 0.0018, 0.0019, 0.0022,
         0.0023], grad_fn=<SelectBackward0>))

In [37]:
tokens[0][0][2:5] = model.lm.special_token_id
sampled_tokens, sampled_probs = sample_argmax(tokens, conditions)
sampled_tokens[0][1].t(), sampled_probs[0][1]

(tensor([[ 342, 1831,  156,  378,  580, 1398, 1731, 1503, 1398, 1728]]),
 tensor([0.0023, 0.0026, 0.0023, 0.0018, 0.0024, 0.0024, 0.0018, 0.0018, 0.0021,
         0.0026], grad_fn=<SelectBackward0>))